In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-fine-food-reviews/hashes.txt
/kaggle/input/amazon-fine-food-reviews/Reviews.csv
/kaggle/input/amazon-fine-food-reviews/database.sqlite


In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,LancasterStemmer,SnowballStemmer,WordNetLemmatizer
import sklearn
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import gensim
from gensim.models import KeyedVectors,Word2Vec

In [3]:
data=pd.read_csv(r"/kaggle/input/amazon-fine-food-reviews/Reviews.csv")

In [4]:
df=data.copy()

In [5]:
df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568428 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [7]:
df.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [8]:
data1=pd.read_csv(r"/kaggle/input/amazon-fine-food-reviews/Reviews.csv",usecols=['Score',"Text"])

In [9]:
dff=data1.copy()

In [10]:
dff

,Score,Text
0,5,I have bought several of the Vitality canned d...
1,1,Product arrived labeled as Jumbo Salted Peanut...
2,4,This is a confection that has been around a fe...
3,2,If you are looking for the secret ingredient i...
4,5,Great taffy at a great price. There was a wid...
...,...,...
568449,5,Great for sesame chicken..this is a good if no...
568450,2,I'm disappointed with the flavor. The chocolat...
568451,5,"These stars are small, so you can give 10-15 o..."
568452,5,These are the BEST treats for training and rew...


In [11]:
class_lab=dff[['Score']]


In [12]:
feature_var=dff[['Text']]

In [13]:
import re

In [14]:
def edat(data,name):
    case =" ".join(data[name]).islower()
    html=data[name].apply(lambda x: True if re.search("<.+?>",x) else False ).sum()
    url=data[name].apply(lambda x:True if re.search("http[s]?://.+? +",x)else False).sum()
    un_wanted=data[name].apply(lambda x:True if re.search("[]()*\-.,@#$%^&0-9]",x)else False ).sum()
    if case == False :
        print("not in lower")
    else :
        print('no')
    if html>0:
        print("have html tages")
    else:
        print('no')
    if  url> 0:
        print("have urls")
    else:
        print("no urls")
    
    if un_wanted >0:
        print("have wanted ")
    else:
        print("no unwanted ")
                               

In [15]:
edat(feature_var,"Text")

not in lower
have html tages
have urls
have wanted 


In [16]:
def textpp(x):
    x = x.lower()
    x = re.sub("[]:()*\-.,@#$%^&0-9]", " ", x)
    x = re.sub("<.+?>", " ", x)
    x = re.sub("http[s]?://.+? +", " ", x)
    return x

In [17]:
feature_var["Text"] =feature_var["Text"].apply(textpp)

/tmp/ipykernel_19/1416528193.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_var["Text"] =feature_var["Text"].apply(textpp)


In [18]:
feature_var["Text"]

0         i have bought several of the vitality canned d...
1         product arrived labeled as jumbo salted peanut...
2         this is a confection that has been around a fe...
3         if you are looking for the secret ingredient i...
4         great taffy at a great price   there was a wid...
                                ...                        
568449    great for sesame chicken  this is a good if no...
568450    i'm disappointed with the flavor  the chocolat...
568451    these stars are small  so you can give       o...
568452    these are the best treats for training and rew...
568453    i am very satisfied  product is as advertised ...
Name: Text, Length: 568454, dtype: object

In [19]:
dff["Text"] = dff["Text"].apply(textpp)

In [20]:
dff

,Score,Text
0,5,i have bought several of the vitality canned d...
1,1,product arrived labeled as jumbo salted peanut...
2,4,this is a confection that has been around a fe...
3,2,if you are looking for the secret ingredient i...
4,5,great taffy at a great price there was a wid...
...,...,...
568449,5,great for sesame chicken this is a good if no...
568450,2,i'm disappointed with the flavor the chocolat...
568451,5,these stars are small so you can give o...
568452,5,these are the best treats for training and rew...


In [21]:
dff1=dff.copy()

In [22]:
dff1

,Score,Text
0,5,i have bought several of the vitality canned d...
1,1,product arrived labeled as jumbo salted peanut...
2,4,this is a confection that has been around a fe...
3,2,if you are looking for the secret ingredient i...
4,5,great taffy at a great price there was a wid...
...,...,...
568449,5,great for sesame chicken this is a good if no...
568450,2,i'm disappointed with the flavor the chocolat...
568451,5,these stars are small so you can give o...
568452,5,these are the best treats for training and rew...


In [23]:
tf=TfidfVectorizer(ngram_range=(1,1))
ttf = tf.fit_transform(dff1["Text"])
ttf

<568454x107252 sparse matrix of type '<class 'numpy.float64'>'
	with 30198754 stored elements in Compressed Sparse Row format>

In [24]:
tf.vocabulary_

{'have': 43073,
 'bought': 10935,
 'several': 84242,
 'of': 65292,
 'the': 94891,
 'vitality': 102236,
 'canned': 13736,
 'dog': 27770,
 'food': 36247,
 'products': 73973,
 'and': 3675,
 'found': 36693,
 'them': 94964,
 'all': 2632,
 'to': 96099,
 'be': 7838,
 'good': 40226,
 'quality': 75571,
 'product': 73955,
 'looks': 55053,
 'more': 60947,
 'like': 54114,
 'stew': 90086,
 'than': 94820,
 'processed': 73863,
 'meat': 58092,
 'it': 49187,
 'smells': 86786,
 'better': 8880,
 'my': 62064,
 'labrador': 52518,
 'is': 49042,
 'finicky': 35061,
 'she': 84571,
 'appreciates': 4663,
 'this': 95279,
 'most': 61125,
 'arrived': 5241,
 'labeled': 52476,
 'as': 5401,
 'jumbo': 50453,
 'salted': 81707,
 'peanuts': 69101,
 'were': 103731,
 'actually': 1030,
 'small': 86697,
 'sized': 86043,
 'unsalted': 100102,
 'not': 64076,
 'sure': 92219,
 'if': 46264,
 'was': 103103,
 'an': 3556,
 'error': 31711,
 'or': 66255,
 'vendor': 101470,
 'intended': 48332,
 'represent': 78846,
 'confection': 19861,
 

In [25]:
tf1=TfidfVectorizer(ngram_range=(1,1),max_features=25)
tf1.fit_transform(dff1["Text"])

<568454x25 sparse matrix of type '<class 'numpy.float64'>'
	with 6451201 stored elements in Compressed Sparse Row format>

In [26]:
tf1.fit_transform(dff1['Text']).toarray()

array([[0.45389198, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.66928902, ..., 0.31898988, 0.        ,
        0.        ],
       [0.43398574, 0.1235599 , 0.        , ..., 0.        , 0.34383709,
        0.12427218],
       ...,
       [0.13550757, 0.23148182, 0.        , ..., 0.        , 0.21471916,
        0.69844868],
       [0.40434808, 0.34536533, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.20861705, 0.        , 0.77010036, ..., 0.        , 0.33056515,
        0.        ]])